# Exercise 5: Memory Protection Mechanisms (Bounds checking, CFI, SFI)

### Assignment Deadline: May 5 (Monday) before 11:59PM

**This assignment is worth 15% of the semester grade.**

In our lecture, we learn about multiple compiler-based, application-level mitigations, including bounds checking, control-flow integrity (CFI), and Software Fault Isolation (SFI). In this exercise, we will ask you to implement the checking logics of these mitigations, **without** dealing with the hassle of static analysis and binary instrumentation. We will start with building an **offline** reference monitor to track all the memory traces and control transfers, and then implement the checking logics corresponding to bounds checking, CFI, and SFI.

## Step 0: Program Tracing

In this step, we will start by collecting the traces of memory access (read and write) and control transfer (call, jump, and return) in a program. Normally, a reference monitor will collect and check the traces online during the program execution. This is important as any operations violating the safety policies will have to be interrupted immediately. However, performing online checks require instrumentation and lots of optimizations to avoid significant overheads. Instead, in this exercise, we will design an **offline** reference monitor, which checks the traces **after** the program execution is finished.

We will use **Pin Tool**, a binary instrumentation and analysis framwork to collect the traces from a program. Pin allows you to design your own binary analyzer, by hooking different logics to operations in the program. Here, we provide you the code for tracing memory access and control transfer, a tool called **memtrace**, which you do not have to develop yourself.

Now, start with downloading the Pin tool:

In [83]:
!wget -O pintool-3.31.tar.gz https://github.com/chiache/csce713-assignments/raw/refs/heads/master/lab5/pintool-3.31.tar.gz
!tar -xzf pintool-3.31.tar.gz

--2025-04-29 05:03:27--  https://github.com/chiache/csce713-assignments/raw/refs/heads/master/lab5/pintool-3.31.tar.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chiache/csce713-assignments/refs/heads/master/lab5/pintool-3.31.tar.gz [following]
--2025-04-29 05:03:27--  https://raw.githubusercontent.com/chiache/csce713-assignments/refs/heads/master/lab5/pintool-3.31.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32977372 (31M) [application/octet-stream]
Saving to: ‘pintool-3.31.tar.gz’

pintool-3.31.tar.gz 100%[===================>]  31.45M   142MB/s    in 0.2s    

2025-04-29 05:03:

Next, examine `memtrace.cpp`, a custom-made Pin tool for tracing memory access and control transfer.

In [84]:
%%writefile pintool-3.31/source/tools/MyPinTool/memtrace.cpp
#include "pin.H"
#include <iostream>
#include <fstream>
#include <thread>

std::ofstream TraceFile;

VOID RecordMemRead(VOID *ip, VOID *addr, THREADID threadID) {
    TraceFile << threadID << "," << ip << ",READ," << addr << "\n";
}

VOID RecordMemWrite(VOID *ip, VOID *addr, THREADID threadID) {
    TraceFile << threadID << "," <<  ip << ",WRITE," << addr << "\n";
}

VOID RecordDirectControlFlow(VOID* ip, VOID* target, const char* type, THREADID threadID) {
    TraceFile << threadID << "," <<  ip << "," << type << "," << target << "\n";
}

VOID RecordIndirectControlFlow(VOID* ip, VOID* target, const char* type, THREADID threadID) {
    TraceFile << threadID << "," <<  ip << "," << type << "," << target << "\n";
}

VOID Instruction(INS ins, VOID *v) {
    if (INS_IsMemoryRead(ins)) {
        INS_InsertPredicatedCall(
            ins, IPOINT_BEFORE, (AFUNPTR)RecordMemRead,
            IARG_INST_PTR,
            IARG_MEMORYREAD_EA,
            IARG_THREAD_ID,
            IARG_END);
    }

    if (INS_IsMemoryWrite(ins)) {
        INS_InsertPredicatedCall(
            ins, IPOINT_BEFORE, (AFUNPTR)RecordMemWrite,
            IARG_INST_PTR,
            IARG_MEMORYWRITE_EA,
            IARG_THREAD_ID,
            IARG_END);
    }

    if (INS_IsRet(ins)) {
        INS_InsertPredicatedCall(
            ins, IPOINT_BEFORE, (AFUNPTR)RecordIndirectControlFlow,
            IARG_INST_PTR,
            IARG_BRANCH_TARGET_ADDR,
            IARG_PTR, "RET",
            IARG_THREAD_ID,
            IARG_END);
    } else if (INS_IsControlFlow(ins)) {
        const char* type = INS_IsCall(ins) ? "CALL" : "JMP";

        if (INS_IsDirectControlFlow(ins)) {
            ADDRINT target = INS_DirectControlFlowTargetAddress(ins);
            INS_InsertPredicatedCall(
                ins, IPOINT_BEFORE, (AFUNPTR)RecordDirectControlFlow,
                IARG_INST_PTR,
                IARG_ADDRINT, target,
                IARG_PTR, type,
                IARG_THREAD_ID,
                IARG_END);
        } else if (INS_IsIndirectControlFlow(ins)) {
            INS_InsertPredicatedCall(
                ins, IPOINT_BEFORE, (AFUNPTR)RecordIndirectControlFlow,
                IARG_INST_PTR,
                IARG_BRANCH_TARGET_ADDR,
                IARG_PTR, type,
                IARG_THREAD_ID,
                IARG_END);
        }
    }
}

// Called when the application exits
VOID Fini(INT32 code, VOID *v) {
    TraceFile.close();
}

// Initialization
int main(int argc, char *argv[]) {
    PIN_Init(argc, argv);
    TraceFile.open("memtrace.out");

    INS_AddInstrumentFunction(Instruction, 0);
    PIN_AddFiniFunction(Fini, 0);

    PIN_StartProgram(); // Never returns
    return 0;
}

Overwriting pintool-3.31/source/tools/MyPinTool/memtrace.cpp


Once you have save the source file, build the custom Pin tool for x86-64:

In [85]:
!cd pintool-3.31/source/tools/MyPinTool && mkdir -p obj-intel64 && make PIN_ROOT=/content/pintool-3.31 obj-intel64/memtrace.so

g++ -Wall -Werror -Wno-unknown-pragmas -DPIN_CRT=1 -fno-stack-protector -fno-exceptions -funwind-tables -fasynchronous-unwind-tables -fno-rtti -DTARGET_IA32E -DHOST_IA32E -fPIC -DTARGET_LINUX -fabi-version=2 -faligned-new -I/content/pintool-3.31/source/include/pin -I/content/pintool-3.31/source/include/pin/gen -isystem /content/pintool-3.31/extras/cxx/include -isystem /content/pintool-3.31/extras/crt/include -isystem /content/pintool-3.31/extras/crt/include/arch-x86_64 -isystem /content/pintool-3.31/extras/crt/include/kernel/uapi -isystem /content/pintool-3.31/extras/crt/include/kernel/uapi/asm-x86 -I/content/pintool-3.31/extras/components/include -I/content/pintool-3.31/extras/xed-intel64/include/xed -I/content/pintool-3.31/source/tools/Utils -I/content/pintool-3.31/source/tools/InstLib -O3 -fomit-frame-pointer -fno-strict-aliasing  -Wno-dangling-pointer -c -o obj-intel64/memtrace.o memtrace.cpp
g++ -shared -Wl,--hash-style=sysv /content/pintool-3.31/intel64/runtime/pincrt/crtbeginS.o

Now, you should be able to run any x86-64 program under this custom pin tool. For example, running `/bin/ls`:


In [86]:
!pintool-3.31/pin -t pintool-3.31/source/tools/MyPinTool/obj-intel64/memtrace.so -- /bin/ls

buffer-overflow    pintool-3.31		sfi-violation	 stack-smashing.c
buffer-overflow.c  pintool-3.31.tar.gz	sfi-violation.c
memtrace.out	   sample_data		stack-smashing


The traces are stored in `memtrace.out`. You may open the file to examine the traces.

## Step 1: Bounds Checking

**Bounds checking** is a classic technique to detect and prevent out-of-bound pointer references from their sources. For each pointer or memory access in program, we can define the upper bound and lower bound of the virtual address, to ensure that the pointer dereferencing or memory access will never go outside of its normal bounds.



Let's start with a simple example of buffer overflow:

In [87]:
%%writefile buffer-overflow.c
#include <stdio.h>
#include <string.h>

char buffer[10];

void vuln_func() {
    char *b = buffer, c;
    while(c = getchar(), c != '\n')
      *(b++) = c;
    printf("You entered: %s\n", buffer);
}

int main() {
    vuln_func();
    return 0;
}

Overwriting buffer-overflow.c


Now, let's compile the program and examine the program binary:

In [88]:
!gcc -o buffer-overflow buffer-overflow.c -no-pie
!objdump -S buffer-overflow


buffer-overflow:     file format elf64-x86-64


Disassembly of section .init:

0000000000401000 <_init>:
  401000:	f3 0f 1e fa          	endbr64 
  401004:	48 83 ec 08          	sub    $0x8,%rsp
  401008:	48 8b 05 e9 2f 00 00 	mov    0x2fe9(%rip),%rax        # 403ff8 <__gmon_start__@Base>
  40100f:	48 85 c0             	test   %rax,%rax
  401012:	74 02                	je     401016 <_init+0x16>
  401014:	ff d0                	call   *%rax
  401016:	48 83 c4 08          	add    $0x8,%rsp
  40101a:	c3                   	ret    

Disassembly of section .plt:

0000000000401020 <.plt>:
  401020:	ff 35 e2 2f 00 00    	push   0x2fe2(%rip)        # 404008 <_GLOBAL_OFFSET_TABLE_+0x8>
  401026:	f2 ff 25 e3 2f 00 00 	bnd jmp *0x2fe3(%rip)        # 404010 <_GLOBAL_OFFSET_TABLE_+0x10>
  40102d:	0f 1f 00             	nopl   (%rax)
  401030:	f3 0f 1e fa          	endbr64 
  401034:	68 00 00 00 00       	push   $0x0
  401039:	f2 e9 e1 ff ff ff    	bnd jmp 401020 <_init+0x20>
  40103f:	90             

You can start with running the program directly under Pin tool with the memory trace module we provided. The program should output a file called ``memtrace.out` which contains all the traces of memory access and control transfer.

In [89]:
!echo "aaaaaaaaaaaaaaa" | pintool-3.31/pin -t pintool-3.31/source/tools/MyPinTool/obj-intel64/memtrace.so -- ./buffer-overflow

You entered: aaaaaaaaaaaaaaa


Now, please design a Python module to perform bounds checking on the memory traces to capture the instruction address(es) where the buffer overflow happens.

First, let's start with defining the bounds checking rules:

In [90]:
raw_bounds_rules = {
    #Each rule should be the format of "IP address: (lower bound, upper bound)"
    #write at IP 0x40117f must stay within buffer [0x404040..0x404049]
    '0x40117f': ('0x404040', '0x404049'),
}

Next, we will read the memory trace output and check the recorded memory traces against the bounds checking rules.

In [91]:
def read_memtrace():
    records = []

    with open('memtrace.out', 'r') as f:
        for line_number, line in enumerate(f, start=1):
            line = line.strip()
            if not line or line.startswith('#'):
                continue  # skip empty lines or comments

            parts = line.split(',')
            if len(parts) != 4:
                print(f"Skipping malformed line {line_number}: {line}")
                continue

            try:
                thread_id = int(parts[0])
                ip_address = int(parts[1], 16)
                op_type = parts[2]
                target_address = int(parts[3], 16)

                record = {
                    'thread_id': thread_id,
                    'ip_address': ip_address,
                    'op_type': op_type,
                    'target_address': target_address
                }
                records.append(record)

            except ValueError as e:
                print(f"Error parsing line {line_number}: {e}")
                continue

    return records

Now, please use the memtrace and the defined bounds checking rules to print out the IP addresses, operation types (read or write), and target addresses of memory safety violation.

In [92]:
read_memtrace()[:5]

[{'thread_id': 0,
  'ip_address': 20998136545939,
  'op_type': 'WRITE',
  'target_address': 140725599402488},
 {'thread_id': 0,
  'ip_address': 20998136545939,
  'op_type': 'CALL',
  'target_address': 20998136549424},
 {'thread_id': 0,
  'ip_address': 20998136549428,
  'op_type': 'WRITE',
  'target_address': 140725599402480},
 {'thread_id': 0,
  'ip_address': 20998136549432,
  'op_type': 'WRITE',
  'target_address': 140725599402472},
 {'thread_id': 0,
  'ip_address': 20998136549434,
  'op_type': 'WRITE',
  'target_address': 140725599402464}]

In [93]:
# --- Normalize into integer‐to‐(low,high) dict ---
bounds_checking_rules = {}
for raw_ip, (raw_low, raw_high) in raw_bounds_rules.items():
    # convert key
    ip = int(raw_ip, 16) if isinstance(raw_ip, str) else raw_ip
    # convert bounds
    low  = int(raw_low,  16) if isinstance(raw_low, str)  else raw_low
    high = int(raw_high, 16) if isinstance(raw_high, str)  else raw_high
    bounds_checking_rules[ip] = (low, high)

In [94]:
bounds_checking_rules

{4198783: (4210752, 4210761)}

In [95]:
def check_bounds(records):
    """
    Walks through records and for any record.ip that has a rule,
    checks if target_address ∈ [low..high].  Prints violations.
    """
    for rec in records:
        ip   = rec['ip_address']
        tgt  = rec['target_address']
        op   = rec['op_type']
        if ip in bounds_checking_rules:
            low, high = bounds_checking_rules[ip]
            if not (low <= tgt <= high):
                print(f"Violation @ IP {hex(ip)}: {op} to {hex(tgt)} "
                      f"outside [{hex(low)}, {hex(high)}]")

In [96]:
recs = read_memtrace()
check_bounds(recs)

Violation @ IP 0x40117f: WRITE to 0x40404a outside [0x404040, 0x404049]
Violation @ IP 0x40117f: WRITE to 0x40404b outside [0x404040, 0x404049]
Violation @ IP 0x40117f: WRITE to 0x40404c outside [0x404040, 0x404049]
Violation @ IP 0x40117f: WRITE to 0x40404d outside [0x404040, 0x404049]
Violation @ IP 0x40117f: WRITE to 0x40404e outside [0x404040, 0x404049]


## Step 2: Control-Flow Integrity

In this step, we will implement a simple version of **control-flow integrity**, a security policy to check the target of calls (both indirect and direct) and returns against a list of valid targets. To start, we will use the same memory trace Pin tool to collect the targets of control flow transfer and check against our rules.

Let's start with a simple example of stack smashing:



In [97]:
%%writefile stack-smashing.c
#include <stdio.h>
#include <string.h>

void attack_func() {
    printf("Stack smashed! You've gained unauthorized access!\n");
}

void vuln_func() {
    char buffer[10];

    printf("Enter some input: ");
    scanf("%s", buffer); // this  is unsafe and allows stack smashing

    printf("You entered: %s\n", buffer);
}

int main() {
    vuln_func();
    printf("Normal execution continues...\n");
    return 0;
}

Overwriting stack-smashing.c


In [98]:
!gcc -fno-stack-protector -no-pie -o stack-smashing stack-smashing.c
!objdump -S stack-smashing


stack-smashing:     file format elf64-x86-64


Disassembly of section .init:

0000000000401000 <_init>:
  401000:	f3 0f 1e fa          	endbr64 
  401004:	48 83 ec 08          	sub    $0x8,%rsp
  401008:	48 8b 05 e9 2f 00 00 	mov    0x2fe9(%rip),%rax        # 403ff8 <__gmon_start__@Base>
  40100f:	48 85 c0             	test   %rax,%rax
  401012:	74 02                	je     401016 <_init+0x16>
  401014:	ff d0                	call   *%rax
  401016:	48 83 c4 08          	add    $0x8,%rsp
  40101a:	c3                   	ret    

Disassembly of section .plt:

0000000000401020 <.plt>:
  401020:	ff 35 e2 2f 00 00    	push   0x2fe2(%rip)        # 404008 <_GLOBAL_OFFSET_TABLE_+0x8>
  401026:	f2 ff 25 e3 2f 00 00 	bnd jmp *0x2fe3(%rip)        # 404010 <_GLOBAL_OFFSET_TABLE_+0x10>
  40102d:	0f 1f 00             	nopl   (%rax)
  401030:	f3 0f 1e fa          	endbr64 
  401034:	68 00 00 00 00       	push   $0x0
  401039:	f2 e9 e1 ff ff ff    	bnd jmp 401020 <_init+0x20>
  40103f:	90              

Now, a small exercise for recaping what we learned in assignment 2. Please come up with an input to force the progam `stack-smashing` to print out the following:

```
Stack smashed! You've gained unauthorized access!
```

In [99]:
!echo "aaaaaaa" | pintool-3.31/pin -t pintool-3.31/source/tools/MyPinTool/obj-intel64/memtrace.so -- ./stack-smashing

Enter some input: You entered: aaaaaaa
Normal execution continues...


In [100]:
!python3 -c 'import sys; sys.stdout.buffer.write(b"A"*18 + b"\x76\x11\x40\x00\x00\x00\x00\x00")' | pintool-3.31/pin -t pintool-3.31/source/tools/MyPinTool/obj-intel64/memtrace.so -- ./stack-smashing

Enter some input: You entered: AAAAAAAAAAAAAAAAAAv@
Stack smashed! You've gained unauthorized access!
/bin/bash: line 1:  8934 Done                    python3 -c 'import sys; sys.stdout.buffer.write(b"A"*18 + b"\x76\x11\x40\x00\x00\x00\x00\x00")'
      8935 Segmentation fault      (core dumped) | pintool-3.31/pin -t pintool-3.31/source/tools/MyPinTool/obj-intel64/memtrace.so -- ./stack-smashing


Next, run `stack-smashing` with the same input and collect memory traces.

In [101]:
raw_return_rules = {
    # Each rule should be the format of "IP address: [ allowed return targets ]"
    '0x4011e8': ['0x4011fb'],
}
raw_call_rules = {
    # Each rule should be the format of "IP address: [ allowed call targets ]"
    '0x4011f6': ['0x401190'],
    '0x401205': ['0x401060'],
    '0x4011e1' :['0x401070'],
    '0x4011c6': ['0x401080'],
    '0x4011ab': ['0x401070'],
}
raw_jump_rules = {
    # Each rule should be the format of "IP address: [ allowed jump targets ]"
}

# --- 2) Normalize hex-strings → ints ---
def _normalize(rules):
    norm = {}
    for ip, tgts in rules.items():
      #print(ip, tgts)
      ip_int = int(ip,  16) if isinstance(ip, str)  else ip
      norm[ip_int] = [int(tgt,  16) if isinstance(tgt, str)  else tgt for tgt in tgts]
    return norm

return_rules = _normalize(raw_return_rules)
call_rules   = _normalize(raw_call_rules)
jump_rules   = _normalize(raw_jump_rules)

Now, please design a Python module to check control flow integrity on the control transfer traces to capture the instruction address(es) where the control flow hijacking happens.

First, let's start with defining the control transfer rules, incuding the rules for returns, calls, and jumps.

Now, please use the memtrace and the defined control flow integrity rules to print out the IP addresses, operation types (return, call, or jump), and target addresses of control flow integrity violation.

In [102]:
def check_cfi(records):
    for rec in records:
        ip   = rec['ip_address']
        tgt  = rec['target_address']
        op   = rec['op_type']

        #if return_rules[ip] and tgt in return_rules[ip]:
        #if ip > 4000000 and ip < 4500000  and (op == 'JMP'):
          #print(op, ip, hex(ip),tgt, hex(tgt))

        if op == 'RET' and ip in return_rules:
            allowed = return_rules[ip]
            if tgt not in allowed:
                print(f"CFI Violation @ {hex(ip)}: RET → {hex(tgt)} not in {list(map(hex,allowed))}")

        elif op == 'CALL' and ip in call_rules:
            allowed = call_rules[ip]
            if tgt not in allowed:
                print(f"CFI Violation @ {hex(ip)}: CALL → {hex(tgt)} not in {list(map(hex,allowed))}")

        elif op == 'JMP' and ip in jump_rules:
            allowed = jump_rules[ip]
            if tgt not in allowed:
                print(f"CFI Violation @ {hex(ip)}: JMP → {hex(tgt)} not in {list(map(hex,allowed))}")

In [103]:
recs = read_memtrace()
check_cfi(recs)

CFI Violation @ 0x4011e8: RET → 0x401176 not in ['0x4011fb']


## Software Fault Isolation (SFI)

**Software Fault Isolation (SFI)** is a technique to prevent faults in one part of a program from corrupting or interfering with other parts. Typically, it's implemented with memory boundaries and control-flow restrictions. Violating SFI usually means that one "compartment" (or thread/module) can corrupt memory it shouldn't be able to touch.

The rules of SFI are thread-based: For each thread (besides thread 0), the rules need to specify two separate rules, one for code and one for data:

*   For code, any jump, call, and return needs to fall within a code area `(CodeStart, CodeEnd)`.
*   For data, any memory access, either read or write, needs to fall within a data area `(DataStart, DataEnd)`.


Let's start with a simple example of violating SFI:

In [104]:
%%writefile sfi-violation.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <string.h>
#include <unistd.h>

#define BUFFER_SIZE 64

typedef struct {
    char buffer[BUFFER_SIZE];
} ThreadData;

void* thread_func_1(void* arg) {
    ThreadData* data = (ThreadData*)arg;
    printf("Thread 1 writing to its own buffer...\n");
    strcpy(data->buffer, "Thread 1 was here!");

    // Malicious overwrite beyond its own buffer
    printf("Thread 1 now corrupting neighbor's buffer...\n");
    memset((char*)data->buffer + BUFFER_SIZE, 'X', 128);

    return NULL;
}

void* thread_func_2(void* arg) {
    ThreadData* data = (ThreadData*)arg;
    printf("Thread 2 sleeping...\n");
    sleep(2); // Give thread 1 time to corrupt
    printf("Thread 2 buffer content: %s\n", data->buffer);
    return NULL;
}

int main() {
    pthread_t t1, t2;
    ThreadData* t1_data = (ThreadData*)malloc(sizeof(ThreadData));
    ThreadData* t2_data = (ThreadData*)malloc(sizeof(ThreadData));

    // Added this line to get the data range
    printf("t1_data at %p, t2_data at %p\n", (void*)t1_data, (void*)t2_data);

    if (!t1_data || !t2_data) {
        perror("malloc");
        return 1;
    }

    memset(t1_data, 0, sizeof(ThreadData));
    memset(t2_data, 0, sizeof(ThreadData));
    strcpy(t2_data->buffer, "Thread 2's secret data.");

    pthread_create(&t1, NULL, thread_func_1, (void*)t1_data);
    pthread_create(&t2, NULL, thread_func_2, (void*)t2_data);

    pthread_join(t1, NULL);
    pthread_join(t2, NULL);

    free(t1_data);
    free(t2_data);

    return 0;
}

Overwriting sfi-violation.c


In [105]:
def read_memtrace_silent():
    records = []

    with open('memtrace.out', 'r') as f:
        for line_number, line in enumerate(f, start=1):
            line = line.strip()
            if not line or line.startswith('#'):
                continue  # skip empty lines or comments

            parts = line.split(',')
            if len(parts) != 4:
                #print(f"Skipping malformed line {line_number}: {line}")
                continue

            try:
                thread_id = int(parts[0])
                ip_address = int(parts[1], 16)
                op_type = parts[2]
                target_address = int(parts[3], 16)

                record = {
                    'thread_id': thread_id,
                    'ip_address': ip_address,
                    'op_type': op_type,
                    'target_address': target_address
                }
                records.append(record)

            except ValueError as e:
                #print(f"Error parsing line {line_number}: {e}")
                continue

    return records

In [106]:
!gcc -o sfi-violation sfi-violation.c -no-pie -pthread
!./sfi-violation

t1_data at 0x2f8482a0, t2_data at 0x2f8482f0
Thread 1 writing to its own buffer...
Thread 1 now corrupting neighbor's buffer...
Thread 2 sleeping...
Thread 2 buffer content: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXThread 2 buffer content: XXXXXXX's buffer...

free(): invalid size


In [107]:
!gcc -o sfi-violation sfi-violation.c -no-pie
!objdump -S sfi-violation


sfi-violation:     file format elf64-x86-64


Disassembly of section .init:

0000000000401000 <_init>:
  401000:	f3 0f 1e fa          	endbr64 
  401004:	48 83 ec 08          	sub    $0x8,%rsp
  401008:	48 8b 05 e9 2f 00 00 	mov    0x2fe9(%rip),%rax        # 403ff8 <__gmon_start__@Base>
  40100f:	48 85 c0             	test   %rax,%rax
  401012:	74 02                	je     401016 <_init+0x16>
  401014:	ff d0                	call   *%rax
  401016:	48 83 c4 08          	add    $0x8,%rsp
  40101a:	c3                   	ret    

Disassembly of section .plt:

0000000000401020 <.plt>:
  401020:	ff 35 e2 2f 00 00    	push   0x2fe2(%rip)        # 404008 <_GLOBAL_OFFSET_TABLE_+0x8>
  401026:	f2 ff 25 e3 2f 00 00 	bnd jmp *0x2fe3(%rip)        # 404010 <_GLOBAL_OFFSET_TABLE_+0x10>
  40102d:	0f 1f 00             	nopl   (%rax)
  401030:	f3 0f 1e fa          	endbr64 
  401034:	68 00 00 00 00       	push   $0x0
  401039:	f2 e9 e1 ff ff ff    	bnd jmp 401020 <_init+0x20>
  40103f:	90               

Next, run the example to collect the memory trace:

In [113]:
!pintool-3.31/pin -t pintool-3.31/source/tools/MyPinTool/obj-intel64/memtrace.so -- ./sfi-violation

t1_data at 0x3f3fc2a0, t2_data at 0x3f3fc2f0
Thread 1 writing to its own buffer...
Thread 2 sleeping...
Thread 1 now corrupting neighbor's buffer...
Thread 2 buffer content: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXThread 2 buffer content: XXXXXXX's buffer...

free(): invalid size


In order to detect any SFI violation, we need to define the SFI rules (in Python), which should looks like this:

In [117]:
# -----------------------------------------------------------------------------
# SFI rules (filling in the DATAx_START from the printf output)
# -----------------------------------------------------------------------------
CODE_START = 0x401000
CODE_END   = 0x4014a0

DATA1_START = 0x3f3fc2a0
DATA1_END   = DATA1_START + 64

DATA2_START = 0x3f3fc2f0
DATA2_END   = DATA2_START + 64

sfi_rules = {
    1: (CODE_START, CODE_END, DATA1_START, DATA1_END),
    2: (CODE_START, CODE_END, DATA2_START, DATA2_END),
}

# -----------------------------------------------------------------------------
def is_violation(tid, op, target):
    if int(tid) not in sfi_rules:
        # no rules for this thread → treat as safe or skip
        #print("Invalid Thread ID", int(tid))
        return False

    cstart, cend, dstart, dend = sfi_rules[tid]

    if op in ('READ', 'WRITE'):
        # data access must fall in [dstart, dend)
        return not (int(dstart) <= int(target) < int(dend))

    elif op in ('CALL', 'JMP', 'RET'):
        # control‐flow must fall in [cstart, cend)
        return not (int(cstart) <= int(target) < int(cend))

    else:
        # unknown op → treat as violation
        print("Unknow!!")
        return True

#-----------------------------------------------------------------------------
def check_sfi(recs):
  for r in recs:
      tid  = r['thread_id']
      ip   = r['ip_address']
      op   = r['op_type']
      tgt  = r['target_address']

      if is_violation(tid, op, tgt):
          print(f"{tid:2d}  0x{ip:016x}  {op:6s}  0x{tgt:016x}")

# -----------------------------------------------------------------------------
# 5) main driver
# -----------------------------------------------------------------------------

In [118]:
recs = read_memtrace_silent()
check_sfi(recs)

 1  0x00000efe3f659840  JMP     0x00000efe3f65985a
 1  0x00000efe3f659842  JMP     0x00000efe3f659845
 1  0x00000efe3f65984e  WRITE   0x00000efe3f109ef8
 1  0x00000efe3f65984e  CALL    0x00000efe3f5c77d0
 1  0x00000efe3f5c77d4  WRITE   0x00000efe3f109ef0
 1  0x00000efe3f5c77d5  WRITE   0x00000efe3f109ee8
 1  0x00000efe3f5c77dd  WRITE   0x00000efe3f109e68
 1  0x00000efe3f5c77e2  READ    0x00000efe3f10a668
 1  0x00000efe3f5c77eb  WRITE   0x00000efe3f109ed8
 1  0x00000efe3f5c77f2  READ    0x00000efe3f10ac53
 1  0x00000efe3f5c77f9  JMP     0x00000efe3f5c7858
 1  0x00000efe3f5c7858  READ    0x00000efe3f109e68
 1  0x00000efe3f5c7864  READ    0x00000efe3f74cfe8
 1  0x00000efe3f5c786b  WRITE   0x00000efe3f10a5b8
 1  0x00000efe3f5c786f  WRITE   0x00000efe3f109e58
 1  0x00000efe3f5c786f  CALL    0x00000efe3f56d3c0
 1  0x00000efe3f56d3c4  READ    0x00000efe3f74cf70
 1  0x00000efe3f56d3cb  READ    0x00000efe3f74cdf8
 1  0x00000efe3f56d3d2  READ    0x00000efe3f10a5b0
 1  0x00000efe3f56d3d6  READ   

Now, please use the memtrace and the defined SFI rules to print out the IP addresses, operation types (read, write, return, call, or jump), and target addresses of SFI violation.

## Submission

Once you have finished this notebook, click "File > Download > Download as .ipynb" and upload the file to **Assignment 5** on MS Teams.

## Reference

Please cite all the sources if there's any.